# XGBoost regression (multi-node with Dask)

<table>
    <tr>
        <td>
            <img src="../_img/dask-horizontal.svg" width="425">
        </td>
        <td>
            <img src="../_img/xgboost.png" width="300">
        </td>
    </tr>
</table>

This notebook describes how to use Dask to scale training of XGBoost models. For more detailed information, see ["Distributed XGBoost with Dask"](https://xgboost.readthedocs.io/en/latest/tutorials/dask.html) in the XGBoost documentation and ["XGBoost Training with Dask"](https://www.saturncloud.io/docs/tutorials/xgboost/) in Saturn Cloud's documentation.

<hr>

## Initialize A Dask Cluster

This tutorial uses multiple machines to show how to apply more computing resources to machine learning training. This is done with Dask. Saturn Cloud offers managed Dask clusters, which can be provisioned and modified programmatically.

The code below creates a Dask cluster using [`dask-saturn`](https://github.com/saturncloud/dask-saturn), the official Dask client for Saturn Cloud. It creates a cluster with the following specs:

* `n_workers=3` --> 3 machines in the cluster
* `scheduler_size='medium'` --> the Dask scheduler will have 4GB of RAM and 2 CPU cores
* `worker_size='large'` --> each worker machine will have 2 CPU cores and 16GB of RAM

To see a list of possible sizes, run the code below.

In [ ]:
import dask_saturn

dask_saturn.describe_sizes()

The `dask-saturn` code below creates two important objects: a cluster and a client.

* `cluster`: knows about and manages the scheduler and workers
    - can be used to create, resize, reconfigure, or destroy those resources
    - knows how to communicate with the scheduler, and where to find logs and diagnostic dashboards
* `client`: tells the cluster to do things
    - can send work to the cluster
    - can restart all the worker processes
    - can send data to the cluster or pull data back from the cluster

In [ ]:
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(n_workers=n_workers, scheduler_size="medium", worker_size="large")
client = Client(cluster)

If you created your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster in the Saturn UI if you want to get a head start!

In [ ]:
client.wait_for_workers(n_workers=n_workers)

### Monitor Resource Usage

This tutorial aims to teach you how to take advantage of multiple GPUs for data science workflows. To prove to yourself that Dask RAPIDS are utilizing the GPUs, it's important to understand how to monitor that utilization while your code is running. If you already know how to do that, skip to the next section.

Print the `cluster` object in a notebook renders a widget that shows the number of workers, available CPU and memory, and a dashboard link.

In [ ]:
cluster

Click that dashboard link to view some diagnostic information about the Dask cluster. This can be used to view the current resource utilization of workers in the cluster and lots of information about what they're currently working on.

<hr>

## Load data

This example is designed to run quickly with small, relatively inexpensive resources. So let's just load a single month of taxi data for training.

In [ ]:
import dask.dataframe as dd

taxi = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
).sample(frac=0.3, replace=False)

The code below computes the size of this dataset in memory.

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e6} MB")

You can examine the structure of the data with Dask DataFrame commands:

`.head()` = view the first few rows

In [ ]:
taxi.head()

`.dtypes` = list all the columns and the type of data in them

In [ ]:
taxi.dtypes

<hr>

## Prep for Training

Before training a model, we need to transform this dataset into a format that's better-suited to the research question. The function below does that with Dask DataFrame operations.

<details><summary>(click here to learn why data scientists do this)</summary>

**Compute the Target**

The raw data don't contain a column that cleanly describes whether or not a particular ride could be considered a "high" tip. So we need to add one! In this exercise, a "high" tip is one that is greater than 20% of the fare amount.

**Add Features**

Giving a machine learning model a richer description of each training observation improves its ability to describe the relationship between those observations' characteristics and the target. These characteristics are called "features".

For example, instead of giving a model a raw timestamp, it can be valuable to provide multiple derived characteristics like hour of the day and day of the week. It's plausible, for example, that weekend rides might have a different distribution of tips because they tend to be for leisure, where weekday rides might be mostly people travelling for work.

**Remove Unused Features**

If the goal is to produce a model that could predict whether or not a tip will be high *at the beginning of the trip*, then characteristics that can only be known AFTER the tip have to be excluded. For example, you can't know the dropoff time or the type of payment until a ride has concluded.

Such features should be dropped before training.
    
</details>

In [ ]:
def prep_df(df: dd.DataFrame, target_col: str) -> dd.DataFrame:
    """
    Prepare a raw taxi dataframe for training.
        * computes the target ('high_tip')
        * adds features
        * removes unused features
    """
    numeric_feat = [
        "pickup_weekday",
        "pickup_weekofyear",
        "pickup_hour",
        "pickup_week_hour",
        "pickup_minute",
        "passenger_count",
    ]
    categorical_feat = [
        "PULocationID",
        "DOLocationID",
    ]
    features = numeric_feat + categorical_feat
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df[target_col] = df.tip_amount / df.fare_amount

    df["pickup_weekday"] = df.tpep_pickup_datetime.dt.weekday
    df["pickup_weekofyear"] = df.tpep_pickup_datetime.dt.isocalendar().week
    df["pickup_hour"] = df.tpep_pickup_datetime.dt.hour
    df["pickup_week_hour"] = (df.pickup_weekday * 24) + df.pickup_hour
    df["pickup_minute"] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [target_col]].astype(float).fillna(-1)

    return df

Run the code below to get a new data frame, `taxi_train`, that can be used directly for model training.

In [ ]:
target_col = "tip_fraction"
taxi_train = prep_df(taxi, target_col)

Dask performs computations in a [lazy manner](https://tutorial.dask.org/01x_lazy.html), so we persist the dataframe to perform data loading and feature processing.

In [ ]:
%%time
taxi_train = taxi_train.persist()
_ = wait(taxi_train)

taxi_train.head()

`taxi_train` is a Dask DataFrame that will be passed in to a machine learning model. Since this is a binary classification task, before proceeding we should examine the distributions of 1s and 0s in the target. This can be done with by combining `.groupby()` and `.count()`

<details><summary>(click here to learn why data scientists do this)</summary>

**Target Distribution**
    
In binary classification tasks, we ask machine learning models to learn the difference between records that produced a 0 for some target and those that produce a 1. In this example:
    
* `high_tip = 1`: "this trip resulted in a high tip"
* `high_tip = 0`: "this trip did not result in a high tip"
    
The "distribution" of this target just means "what % of the trips ended in a high tip?". This is important to understand because we might have to change our modelling approach if the distribution was uneven. For example, if 99% of trips did NOT end in a high tip, a model that just always guessed "not a high tip" would achieve an accuracy of 99%.

</details>

In [ ]:
taxi_train.groupby("high_tip")["high_tip"].count().compute()

Before going further, check the first few rows of the dataset to make sure that the features look reasonable.

Now that the dataframe has been processed, check its size in memory again.

In [ ]:
print(
    f"Num rows: {len(taxi_train)}, Size: {taxi_train.memory_usage(deep=True).sum().compute() / 1e9} GB"
)

# Train a model

This example uses the native Dask integration built into XGBoost. That integration was added in `xgboost` 1.3.0, and should be preferred to [`dask-xgboost`](https://github.com/dask/dask-xgboost).

In [ ]:
import xgboost as xgb

Training data for `xgboost.dask` needs to be prepared in a special object called `DaskDMatrix`. This is like the XGBoost `DMatrix` that you might be familiar with, but is backed by Dask's distributed collections (Dask DataFrame and Dask Array).

In [ ]:
features = [c for c in taxi_train.columns if c != target_col]

data = taxi_train[features]
label = taxi_train[y_col]

dtrain = xgb.dask.DaskDMatrix(client=client, data=data, label=label)

You can pass any [xgboost parameters](https://xgboost.readthedocs.io/en/latest/parameter.html) to `xgb.dask.train()`. The training process will then start up on all workers that have some of the data in `dtrain`.

In [ ]:
%%time
result = xgb.dask.train(
    client=client,
    params={
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "learning_rate": 0.1,
        "max_depth": 5,
    },
    dtrain=dtrain,
    num_boost_round=50,
)

<hr>

## Save model

`xgb.dask.train()` produces a regular `xgb.core.Booster` object, the same model object produced by non-Dask training.

In [ ]:
booster = result["booster"]
type(booster)

# xgboost.core.Booster

Once you've trained a model, save it in a file to use later for scoring or for comparison with other models.

There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/xgboost_dask.pkl", "wb") as f:
    cloudpickle.dump(booster, f)

<hr>

## Calculate metrics on test set

Use a different month for test set

In [ ]:
taxi_test = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-02.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
).sample(frac=0.01, replace=False)

taxi_test = prep_df(taxi_test)

`xgboost.dask.predict()` can be used to create predictions on a Dask collection using an XGBoost model object. Because the model object here is just a regular XGBoost model, using `xgboost.dask` for batch scoring doesn't require that you also perform training on Dask.

This function returns a Dask Array or Dask Series of predictions, depending on the input type.

In [ ]:
preds = xgb.dask.predict(client=client, model=booster, data=taxi_test[features])

The metrics functions in `dask_ml` can compute metrics on Dask collections like Dask Array and Dask DataFrame, so you never have to hold all of the test data in memory on the client. These functions intentionally mimic the metrics functions in scikit-learn.

In [ ]:
from dask_ml.metrics import mean_squared_error

mean_squared_error(taxi_test[y_col].to_dask_array(), preds.to_dask_array(), squared=False)

<hr>

## Next Steps

In this tutorial, you learned how to use Dask and `xgboost` for distributed model training.

If you want to try working with larger datasets, change the code in this notebook to use more data or explore how to programmatically resize your Dask cluster by following the tips in ["Managing Dask Resources for Machine Learning Training"](https://www.saturncloud.io/docs/tips-and-tricks/training-on-dask).

Alternatively, try [this notebook](./dashboard.ipynb) to learn how to use Saturn Cloud to deploy an interactive dashboard that includes a component which uses the models trained above.

<hr>